## Finding the global minimum of the Rosenbrock function
---

>In mathematical optimization, the Rosenbrock function is a non-convex function, introduced by Howard H. Rosenbrock in 1960, which is used as a performance test problem for optimization algorithms. It is also known as Rosenbrock's valley or Rosenbrock's banana function.
>
>The global minimum is inside a long, narrow, parabolic shaped flat valley. To find the valley is trivial. To converge to the global minimum, however, is difficult.

<p style='text-align: right;'> -- <a href=https://en.wikipedia.org/wiki/Rosenbrock_function>Wikipedia</a> </p>

### The OpenMDAO way
---
#### Create a `Component` to represent the Rosenbrock function

In [1]:
import numpy as np
import openmdao.api as om
    
import time
    
ORDER = 6  # dimension of problem

span = 2   # upper and lower limits
lower_bound = -span*np.ones(ORDER)
upper_bound = span*np.ones(ORDER)

class RosenbrockComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', np.zeros(ORDER))
        self.add_output('y', 0.0)

    def compute(self, inputs, outputs):
        x = inputs['x']

        n = len(x)
        s = 0
        for i in range(n - 1):
            s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
        
        outputs['y'] = s

#### Create a model by putting the `Component` in a `Group` and specifying the design variables and objective

In [2]:
my_model = om.Group()
my_model.add_subsystem('rosenbrock', RosenbrockComp())
my_model.add_design_var('rosenbrock.x', lower=lower_bound, upper=upper_bound)
my_model.add_objective('rosenbrock.y')

#### Create a `Problem` to optimize the model using your choice of optimizers as the `Driver`

In [3]:
p = om.Problem(model=my_model, driver=om.DifferentialEvolutionDriver(max_gen=540))
p.setup()

fail = p.run_driver()
assert fail is False

#### The global minimum is found at [1, 1, 1, ...]

In [4]:
print('xopt:', p['rosenbrock.x'])
print('fopt:', p['rosenbrock.y'])
print('component execution count:', p.driver.iter_count)

xopt: [0.99990757 0.99984523 0.99973422 0.99956889 0.999139   0.99817331]
fopt: [3.43130787e-06]
component execution count: 64921





### The `pycma` way
---
#### Using the functional interface

In [5]:
import cma

def rosenbrock(x):
    n = len(x)
    s = 0
    for i in range(n - 1):
        s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
    return s

# NOTE: the Rosenbrock function is also available in the "fitness functions" library
# rosenbrock = cma.ff.rosen

# find global minimum via the fmin function
res = cma.fmin(rosenbrock, ORDER*[-1], 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound]})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=372823, Tue Oct 20 20:18:21 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 1.980156259923752e+03 1.0e+00 1.03e-02  1e-02  1e-02 0:00.0
    2     18 1.945387554048944e+03 1.4e+00 1.13e-02  1e-02  1e-02 0:00.0
    3     27 1.906794849517519e+03 1.4e+00 1.32e-02  1e-02  1e-02 0:00.0
  100    900 7.005204449634220e-01 1.2e+01 7.37e-02  2e-02  5e-02 0:00.4
  200   1800 6.813190901307600e-06 4.6e+01 3.09e-03  1e-04  2e-03 0:00.7
  210   1890 9.418928833575618e-07 6.2e+01 1.32e-03  4e-05  8e-04 0:00.8
termination on ftarget=1e-06 (Tue Oct 20 20:18:22 2020)
final/bestever f-value = 5.542337e-07 5.542337e-07
incumbent solution: [1.0000307882956503, 1.0000181245383568, 1.000072546732594, 1.0001719640673834, 1.0003324422511015, 1.0006689995380043]
std deviation: [3.826397825493073e-05, 6.611720768019783e-05, 0.00011711324964901171, 0.00021630454082316412, 0.00041358021048492277, 0.00083202989063739

#### The global minimum is found at [1, 1, 1, ...]

In [6]:
print('xopt:', res[0])
print('fopt:', res[1])
print('function execution count:',res[2])

xopt: [1.00003079 1.00001812 1.00007255 1.00017196 1.00033244 1.000669  ]
fopt: 5.542336954888784e-07
function execution count: 1891


### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Component` for use with the `pycma` functional interface

In [7]:
comp = RosenbrockComp()

def rosenbrock(x):
    inputs = {'x': x}
    outputs = {}
    comp.compute(inputs, outputs)
    return outputs['y']
    
# find global minimum via the fmin function
res = cma.fmin(rosenbrock, ORDER*[-1], 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound]})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=390251, Tue Oct 20 20:18:22 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 1.991005560525249e+03 1.0e+00 1.04e-02  1e-02  1e-02 0:00.0
    2     18 1.962021100997369e+03 1.4e+00 1.16e-02  1e-02  1e-02 0:00.0
    3     27 1.903089332254380e+03 1.7e+00 1.44e-02  1e-02  2e-02 0:00.0
  100    900 3.583489836286987e+00 5.5e+00 8.40e-02  2e-02  5e-02 0:00.4
  200   1800 1.676598201360907e-01 1.7e+01 1.39e-01  6e-03  3e-02 0:00.8
  272   2448 9.920636555609427e-07 6.2e+01 5.06e-03  6e-05  1e-03 0:01.1
termination on ftarget=1e-06 (Tue Oct 20 20:18:24 2020)
final/bestever f-value = 4.301704e-07 4.301704e-07
incumbent solution: [0.9999856454682604, 0.9999662793402372, 0.9999546735827822, 0.9999358588988609, 0.9998448779698816, 0.999735039932169]
std deviation: [5.641026847122257e-05, 7.20426356658285e-05, 0.00012755901460320763, 0.00024716691890499414, 0.00047627721455559323, 0.000953902616222939

#### The global minimum is found at [1, 1, 1, ...]

In [8]:
print('xopt:', res[0])
print('fopt:', res[1])
print('component execution count:',res[2])

xopt: [0.99998565 0.99996628 0.99995467 0.99993586 0.99984488 0.99973504]
fopt: 4.3017037341572856e-07
component execution count: 2449


#### Wrap an OpenMDAO `Problem` for use with the `pycma` functional interface

In [9]:
p.setup()

def rosenbrock(x):
    p['rosenbrock.x'] = x
    p.run_model()
    return p['rosenbrock.y'][0]
    
# find global minimum via the fmin function
res = cma.fmin(rosenbrock, ORDER*[-1], 0.01,
               options={'ftarget':1e-6, 'bounds':[lower_bound, upper_bound]})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=369840, Tue Oct 20 20:18:24 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 1.980974769141394e+03 1.0e+00 1.01e-02  1e-02  1e-02 0:00.0
    2     18 1.934791668431192e+03 1.3e+00 1.23e-02  1e-02  1e-02 0:00.0
    3     27 1.908353633656837e+03 1.4e+00 1.46e-02  1e-02  2e-02 0:00.1
  100    900 4.633763146129525e+00 1.4e+01 9.98e-02  3e-02  7e-02 0:00.7
  200   1800 3.973941716540090e+00 5.3e+01 2.69e-03  8e-05  1e-03 0:01.4
  284   2556 3.973940500930305e+00 7.9e+01 8.41e-07  4e-09  9e-08 0:02.0
termination on tolfun=1e-11 (Tue Oct 20 20:18:27 2020)
final/bestever f-value = 3.973941e+00 3.973941e+00
incumbent solution: [-0.9865749788588012, 0.9833982253552974, 0.9721066676851607, 0.9474374251852211, 0.8986511657386549, 0.8075739169285647]
std deviation: [3.809677881060278e-09, 6.439919869413269e-09, 1.2419507644642732e-08, 2.5987227608549275e-08, 4.892489681113457e-08, 8.814261108594318e-

#### The global minimum is found at [1, 1, 1, ...]

In [10]:
print('xopt:', res[0])
print('fopt:', res[1])
print('component execution count:',res[2])

xopt: [-0.98657498  0.98339823  0.97210668  0.94743746  0.89865123  0.80757404]
fopt: 3.973940500930301
component execution count: 2508
